# Merging EB terms-  NLS -  Encyclopaedia Britannica


### Loading the necessary libraries

In [1]:
import yaml
import numpy as np
import collections

In [79]:
import pandas as pd
from yaml import safe_load
from pandas.io.json import json_normalize

### Functions

In [80]:
def read_query_results(filename):
    with open('./results_NLS/'+filename, 'r') as f:
        query_results = safe_load(f)
    return query_results


In [81]:
def write_query_results(filename, results):
    with open('./results_NLS/'+filename, 'w') as f:
        documents = yaml.dump(results, f)

In [82]:
def create_dataframe(query_results):
  
    
    for edition in query_results:
        for page in query_results[edition]:
            #print(page[1].keys())
            column_list=list(page[1].keys())
            break
        break
        
    data=[]
    for edition in query_results:
        for page in query_results[edition]:
            try:
                data.append(page[1])
               
            except:
                pass
    df = pd.DataFrame(data, columns = column_list)
    #removing the columns that I dont need 
    df= df.drop(['last_term_in_page', 'model', 'num_articles', 'num_page_words', 'num_text_unit' , 'text_unit', 'type_archive'], axis=1)
    #renaming the page num
    df= df.rename(columns={"text_unit_id": "page_num", "type_page": "type_article"})
    #removing 'Page' from the string
    df["page_num"] = df["page_num"].str.replace("Page", "")
    df["page_num"] = df["page_num"].astype(int)
    df_tmp= df["edition"].str.split("Volume", expand=True)[1].str.split(",", expand=True)
    df["volume"]= df_tmp[0]
    df["letters"] = df_tmp[1]
    df['volume'] = df["volume"].str.replace(" ", "").astype(int)
    df['term'] = df["term"].str.replace("_def", "")
    df['term']= df["term"].str.replace('[^a-zA-Z0-9]', '')
    mask=df["term"].str.isalpha()
    df=df.loc[mask] 
    df['term'] = df['term'].str.upper()
    
    #df['archive'] = df["archive_filename"].str.replace("/ ", "")144133901/
    

    list_editions={"1":["first", "First"], "2":["second", "Second"],\
               "3":["third", "Third"],\
               "4":["fourth", "Fourth"], \
               "5":["fifth","Fifth"], "6":["sixth","Sixth"],\
               "7":["seventh", "Seventh"], "8":["eighth", "Eighth"]} 
    
    for ed in list_editions:
        for ed_versions in list_editions[ed]:
            mask = df["edition"].str.contains(ed_versions)
            df.loc[mask, 'edition_num'] = ed  
    df['edition_num']=df["edition_num"].astype(int)
    a=df["archive_filename"].str.split("/").str[-2]
    df['source_text_file']= a+ "/" + df["source_text_file"]   
    df= df.drop(['edition', 'archive_filename'], axis=1)
    
    
    df = df[["term", "definition", "related_terms", "num_article_words", "header", "page_num", "term_id_in_page", "type_article", "edition_num", "volume", "letters", "year", "title",  "place", "source_text_file"  ]]
    
    df = df[df['term'] != '']
    
    return df

In [83]:
def create_dataframe_from_file(filename):
    with open('./results_NLS/'+filename, 'r') as f:
        query_results = safe_load(f)
    
    df = create_dataframe(query_results)
    return df

In [84]:
def prune_json(json_dict):
    """
    Method that given a JSON object, removes all its empty fields.
    This method simplifies the resultant JSON.
    :param json_dict input JSON file to prune
    :return JSON file removing empty values
    """
    final_dict = {}
    if not (isinstance(json_dict, dict)):
        # Ensure the element provided is a dict
        return json_dict
    else:
        for a, b in json_dict.items():
            if b or isinstance(b, bool):
                if isinstance(b, dict):
                    aux_dict = prune_json(b)
                    if aux_dict:  # Remove empty dicts
                        final_dict[a] = aux_dict
                elif isinstance(b, list):
                    aux_list = list(filter(None, [prune_json(i) for i in b]))
                    if len(aux_list) > 0:  # Remove empty lists
                        final_dict[a] = aux_list
                else:
                    final_dict[a] = b
    return final_dict

In [85]:
def delete_entries(query_results_updated, eliminate_pages):
    new_results={}
    for edition in query_results_updated:
        new_results[edition]=[]
        for page_idx in range(0, len(query_results_updated[edition])):
            if page_idx not in eliminate_pages[edition]:
                new_results[edition].append(query_results_updated[edition][page_idx])
    return new_results

In [86]:
def merge_terms(query_results):
    eliminate_pages={}
    for edition in query_results:
        eliminate_pages[edition]=[]
        page_number_dict={}
        for page_idx in range(0, len(query_results[edition])):
            
            current_page=query_results[edition][page_idx][0]
            if current_page not in page_number_dict:
                page_number_dict[current_page]=page_idx
        
            element = query_results[edition][page_idx][1]
            if "previous_page" in element['term']:
                current_definition= element["definition"]
                previous_page_idx= page_idx -1
                previous_page_number = current_page -1
                num_article_words=element["num_article_words"]
                related_terms=element["related_terms"]
                
                
                prev_elements = query_results[edition][previous_page_idx][1]
                if prev_elements["last_term_in_page"]:
                   
                    prev_elements["definition"]+=current_definition
                    prev_elements["num_article_words"]+=num_article_words
                    prev_elements["related_terms"]+= related_terms
                    prev_number = int(prev_elements['text_unit_id'].split("Page")[1])
                    
                    for prev_articles_idx in range(page_number_dict[prev_number], page_idx):
                       
                        if query_results[edition][prev_articles_idx][0] == prev_number:
                           
                            query_results[edition][prev_articles_idx][1]["num_page_words"]+=num_article_words
                    
                  
                    for update_element_idx in range(page_number_dict[current_page], page_idx+1):
                        if query_results[edition][update_element_idx][0] == current_page:
                            query_results[edition][update_element_idx][1]["num_page_words"]-=num_article_words
                            query_results[edition][update_element_idx][1]["num_articles"]-=1
                    
                
                eliminate_pages[edition].append(page_idx)
              
   
    new_results= delete_entries(query_results, eliminate_pages)
    
    return new_results

### 1. Reading, Merging articles,  and Writing the results in a new file

Here we are going to take the output of the defoe files, and we are going to merge the terms that splitted across pages. 

In [87]:
query_results=read_query_results('results_eb_1_edition')

In [88]:
query_results_updated =merge_terms(query_results)

Once the data has been merged, we are going to store it in a file, just to have the data merged.  

In [89]:
write_query_results("results_eb_1_edition_updated", query_results_updated)

### 2. Creating a dataframe from the updated results

Once, we have the terms properly merged, we are going to create a dataframe, which we will be use later to do further exploration. In this dataframe we have dropped some information from the original defoe files, that we don not longer need. 

**The dataframe will have the following columns**

- definition:           Definition of the article
- edition_num:          1,2,3,4,5,6,7,8
- header:               Header of the page's article                                  
- num_article_words:    Number of words per article
- place:                Place where the volume was edited (e.g. Edinburgh)                                    
- related_terms:        Related articles (see X article)  
- source_text_file:     File Path of the XML file from which the article belongs       
- term:                 Article name                            
- term_id_in_page:      Number of article in the page     
- page_num:                 Page Number
- title:               Title of the Volume
- type_article:            Type of Page [Full Page| Topic| Mix | Articles]                                       
- year:                 Year of the Volume
- volume:               volume (e.g. 1)
- letters:              leters of the volume (A-B)
- edition_num:           1,2,3,4,5,6,7,8


In [90]:
df=create_dataframe(query_results_updated)

In [91]:
df.tail()

,term,definition,related_terms,num_article_words,header,page_num,term_id_in_page,type_article,edition_num,volume,letters,year,title,place,source_text_file
29298,ZWEIBRUGGEN,"a county of the palatinate of the Rhine, in Ge...",[SQALVS],23,ZoDZYG,857,23,Article,1,3,M-Z,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,nls-data-encyclopaediaBritannica/alto/18837502...
29299,ZYGOMA,in anatomy. See Anatomy p. 152.,[],6,ZoDZYG,857,24,Article,1,3,M-Z,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,nls-data-encyclopaediaBritannica/alto/18837502...
29300,ZYGOMATICUS,"in anatomy,. See Anatomy, p. 306,",[ANATOMY],6,ZoDZYG,857,25,Article,1,3,M-Z,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,nls-data-encyclopaediaBritannica/alto/18837502...
29301,ZYGOPHYLLUM,"in botany, a gr.nus of the decanoria monogyma ...",[],48,ZoDZYG,857,26,Article,1,3,M-Z,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,nls-data-encyclopaediaBritannica/alto/18837502...
29302,CORRIGENDA,"page 132. col. I. !. 2J, 26. at frow the faliv...",[],655,CORRIGENDA,858,0,Topic,1,3,M-Z,1773,"Encyclopaedia Britannica: or, A dictionary of ...",London,nls-data-encyclopaediaBritannica/alto/18837503...


### 3. Saving the dataframe to json file 

In [92]:
df.to_json(r'./results_NLS/results_eb_1_edition_postprocess_dataframe', orient="index") 